In [15]:
!pip install -r requirements.txt

In [ ]:
import duckdb
import pandas as pd
import ast


In [ ]:

df = pd.read_csv('data/cleaned_spotify_tracks.csv')

# Convert list-like strings to real lists
df['track_genre'] = df['track_genre'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)


In [18]:
df.head()

,track_id,artists,track_name,track_genre,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,primary_artist,duration_sec,duration_min
0,0000vdREvCVMxbQTkS888c,['Rill'],Lolly,[german],1,44,0.910,0.374,8,-9.844,0,0.1990,0.075700,0.00301,0.1540,0.432,104.042,160725,4,Rill,160.725,2.678750
1,000CC8EParg64OmTxVnZ0p,['Glee Cast'],It's All Coming Back To Me Now (Glee Cast Version),[club],0,47,0.269,0.516,0,-7.361,1,0.0366,0.406000,0.00000,0.1170,0.341,178.174,322933,4,Glee Cast,322.933,5.382217
2,000Iz0K615UepwSJ5z2RE5,"['Paul Kalkbrenner', 'Pig&Dan']",Böxig Leise - Pig & Dan Remix,[minimal-techno],0,22,0.686,0.560,5,-13.264,0,0.0462,0.001140,0.18100,0.1110,0.108,119.997,515360,4,Paul Kalkbrenner,515.360,8.589333
3,000RDCYioLteXcutOjeweY,['Jordan Sandhu'],Teeje Week,[hip-hop],0,62,0.679,0.770,0,-3.537,1,0.1900,0.058300,0.00000,0.0825,0.839,161.721,190203,4,Jordan Sandhu,190.203,3.170050
4,000qpdoc97IMTBvF8gwcpy,['Paul Kalkbrenner'],Tief,[minimal-techno],0,19,0.519,0.431,6,-13.606,0,0.0291,0.000964,0.72000,0.0916,0.234,129.971,331240,4,Paul Kalkbrenner,331.240,5.520667


In [ ]:
pd.set_option("display.width", 2000)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

# Get the top 20 most popular tracks
result = duckdb.query("""
    SELECT track_name, primary_artist, popularity
    FROM df
    ORDER BY popularity DESC
    LIMIT 20
""").to_df()


print("Top 20 most popular tracks")
print(result)
print("/n")

# Select tracks with danceability greater than 0.9
result = duckdb.query("""
SELECT track_name, primary_artist, danceability
FROM df
WHERE danceability > 0.9
ORDER BY danceability DESC;
""").to_df()

print("Tracks with danceability greater than 0.9")
print(result)
print("/n")

# Select average audio features by genre
result = duckdb.query("""
    SELECT 
        g.genre,
        AVG(danceability) AS avg_dance,
        AVG(energy) AS avg_energy,
        AVG(valence) AS avg_valence,
        AVG(tempo) AS avg_tempo
    FROM df
    CROSS JOIN UNNEST(track_genre) AS g(genre)
    GROUP BY g.genre
    ORDER BY avg_energy DESC
""").to_df()


print("Average audio features by genre")
print(result)
print("/n")

# Get the loudest tracks 
result = duckdb.query("""
SELECT track_name, primary_artist, loudness
FROM df
ORDER BY loudness DESC;
""").to_df()

print("Loudest tracks")
print(result)
print("/n")

# Get the 20 longest tracks
result = duckdb.query("""
SELECT track_name, primary_artist, duration_min, duration_sec
FROM df
ORDER BY duration_min DESC
LIMIT 20;
""").to_df()

print("20 Longest tracks")
print(result)
print("/n")

# Get the 20 shortest tracks
result = duckdb.query("""
SELECT track_name, primary_artist, duration_min, duration_sec
FROM df
ORDER BY duration_min ASC
LIMIT 20;
""").to_df()
print("20 Shortest tracks")
print(result)
print("/n")

# Get tracks with instrumentalness greater than 0.8
result = duckdb.query("""
SELECT track_name, primary_artist, instrumentalness
FROM df
WHERE instrumentalness > 0.8
ORDER BY instrumentalness DESC;
""").to_df()

print("Tracks with instrumentalness greater than 0.8")
print(result)
print("/n")

# Get the average popularity by year
result = duckdb.query("""
SELECT release_year, AVG(popularity) AS avg_popularity
FROM df
GROUP BY release_year
ORDER BY release_year ASC;
""").to_df()

print("Average popularity by year")
print(result)
print("/n")

# Get tracks with speechiness greater than 0.5
result = duckdb.query("""
SELECT track_name, primary_artist, speechiness
FROM df
WHERE speechiness > 0.5
ORDER BY speechiness DESC;
""").to_df()

print("Tracks with speechiness greater than 0.5")
print(result)
print("/n")

# Get the top 10 genres by number of tracks
result = duckdb.query("""
    SELECT g.genre, COUNT(*) AS track_count
    FROM df
    CROSS JOIN UNNEST(track_genre) AS g(genre)
    GROUP BY g.genre
    ORDER BY track_count DESC
    LIMIT 10
""").to_df()

print("Top 10 genres by number of tracks")
print(result)


Top 20 most popular tracks
                               track_name     primary_artist  popularity
0               Unholy (feat. Kim Petras)          Sam Smith         100
1   Quevedo: Bzrp Music Sessions, Vol. 52           Bizarrap          99
2                         I'm Good (Blue)       David Guetta          98
3                              La Bachata      Manuel Turizo          98
4                         Me Porto Bonito          Bad Bunny          97
5                        Tití Me Preguntó          Bad Bunny          97
6                     Under The Influence        Chris Brown          96
7                                  Efecto          Bad Bunny          96
8                         I Ain't Worried        OneRepublic          96
9                               As It Was       Harry Styles          95
10                          Ojitos Lindos          Bad Bunny          95
11                            Moscow Mule          Bad Bunny          94
12                      